In [1]:
import requests
import re
import time
import json
from tqdm import tqdm

In [2]:
book_dir = 'vulgate'
page_dir = 'mark_1'

In [3]:
def parse_text(text):
    lines = []
    text = re.split('\.|\?|\!|\:|\;|\,', text)
    
    for line in text:
        line = line.split(' ')
        line = [word for word in line if word != '']
        lines.append(line)

    lines.remove([])
    return lines

In [4]:
def get_punct_marks(text):
    punct_marks = [char for char in text if char not in 'abcdefghijklmnopqurstuvwxyzABCDEFGHIJKLMNOPQURSTUVWXYZ[]()123456789']
    punct_marks = [p for p in punct_marks if p != ' ']
    return punct_marks

In [5]:
def get_whitakers_data(word):
    r = requests.get(f'https://latin-words.com/cgi-bin/translate.cgi?query={word}')
    parsed_response = r.json()['message'].split('\n')
    parsed_response = [x.strip() for x in parsed_response if x != '']
    return parsed_response

In [6]:
with open(f'../texts/{book_dir}/{page_dir}/text_data.txt', 'r') as f:
    text_data = f.read()

In [7]:
parsed_text = parse_text(text_data)
punct_marks = get_punct_marks(text_data)

In [8]:
page_obj = {}
counter = 1
for line, punct in tqdm(zip(parsed_text, punct_marks)):
    for word in line:
        word = word.translate({ord(x): None for x in '[]()123456789'})
        if word != '':
            word_data = get_whitakers_data(word)
            time.sleep(.5)
            page_obj[counter] = {
                "word": word,
                "word_data": word_data
            }
            counter += 1
    page_obj[counter] = {
        "word": punct,
        "word_data": "punct"
    }
    counter += 1

29it [02:39,  5.50s/it]


In [9]:
with open(f'../texts/{book_dir}/{page_dir}/json_data.json', 'w') as outfile:
    json.dump(page_obj, outfile)